## 🤗 Finetune **Longformer Encoder-Decoder (LED)** on 8K Tokens 🤗

The *Longformer Encoder-Decoder (LED)* was recently added as an extension to [Longformer: The Long-Document Transformer](https://arxiv.org/abs/2004.05150) by Iz Beltagy, Matthew E. Peters, Arman Cohan.

In this notebook we will finetune *LED* for Summarization on [Pubmed](https://huggingface.co/datasets/viewer/?dataset=scientific_papers). *Pubmed* is a long-range summarization dataset, which makes it a good candidate for LED. LED will be finetuned up to an input length of 8K tokens on a single GPU.

We will leverage 🤗`Seq2SeqTrainer`, gradient checkpointing and as usual 🤗`datasets`.

First, let's try to get a GPU with at least 15GB RAM.

To check that we are having enough RAM we can run the following command.
If the randomely allocated GPU is too small, the above cells can be run 
to crash the notebook hoping to get a better GPU.

In [43]:
import pandas as pd
from datasets import Dataset
from datasets import load_from_disk
from sklearn.model_selection import train_test_split

In [44]:
from datasets import load_dataset, load_metric
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
)

In [45]:
# %%capture
# !pip install datasets==1.2.1
# !pip install transformers==4.2.0
# !pip install rouge_score

In [46]:
# huggingface datasets load

train_dataset = load_from_disk("/home/aiffelsummabot/LED/HF_train_df/")
val_dataset = load_from_disk("/home/aiffelsummabot/LED/HF_val_df/")
test_dataset = load_from_disk("/home/aiffelsummabot/LED/HF_test_df/")

In [47]:
train_dataset[0]

{'article': ' from outstanding performance in 2011 12, burberry began the year cautiously optimistic , our long-range objectives ensuring clarity of the luxury brand message , enabling sustainable growth and being a great company firmly in sight . angela ahrendts chief executive officer this combination of optimism and determination , fuelled by the brand’s wealth of opportunity , suggested continued pursuit of the investment-oriented strategic agenda in the year ahead . at the same time , this pre-disposition was tempered by uncertainties in the macro environment and the goal to deliver near-term financial performance . in the final analysis , the result was a balance of dynamic management , core execution and strategic investment . challenging context following standout growth in 2011 relative to the range of consumer sectors , luxury slowed dramatically in 2012. the ongoing economic crisis in the eurozone and a continued sluggish us weighed on all areas of consumer spending . althou

In [48]:
train_dataset

Dataset({
    features: ['__index_level_0__', 'abstract', 'article'],
    num_rows: 2445
})

In [49]:
!nvidia-smi

Sun Dec 12 01:30:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    32W / 250W |   1703MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  On   | 00000000:00:05.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |     10MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

Next, we install 🤗Transformers, 🤗Datasets, and `rouge_score`.



Let's start by loading and preprocessing the dataset.



Next, we download the pubmed train and validation dataset ([click to see on 🤗Datasets Hub](https://huggingface.co/datasets/scientific_papers)). This can take a couple of minutes **☕** .

It's always a good idea to take a look at some data samples. Let's do that here.

In [50]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=4):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

We can see that the input data is the `article` - a scientific report and the target data is the `abstract` - a concise summary of the report.

Cool! Having downloaded the dataset, let's tokenize it.
We'll import the convenient `AutoTokenizer` class.

In [51]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

Note that for the sake of this notebook, we finetune the "smaller" LED checkpoint ["allenai/led-base-16384"](https://huggingface.co/allenai/led-base-16384). Better performance can however be attained by finetuning ["allenai/led-large-16384"](https://huggingface.co/allenai/led-large-16384) at the cost of a higher required GPU RAM.

Pubmed's input data has a median token length of 2715 with the 90%-ile token length being 6101. The output data has a media token length of 171 with the 90%-ile token length being 352.${}^1$. 

Thus, we set the maximum input length to 8192 and the maximum output length to 512 to ensure that the model can attend to almost all input tokens is able to generate up to a large enough number of output tokens.

In this notebook, we are only able to train on `batch_size=2` to prevent out-of-memory errors.

---
${}^1$ The data is taken from page 11 of [Big Bird: Transformers for Longer Sequences](https://arxiv.org/pdf/2007.14062.pdf).


In [52]:
max_input_length = 4096
max_output_length = 512
batch_size = 1

Now, let's write down the input data processing function that will be used to map each data sample to the correct model format.
As explained earlier `article` represents here our input data and `abstract` is the target data. The datasamples are thus tokenized up to the respective maximum lengths of 8192 and 512.

In addition to the usual `attention_mask`, LED can make use of an additional `global_attention_mask` defining which input tokens are attended globally and which are attended only locally, just as it's the case of [Longformer](https://huggingface.co/transformers/model_doc/longformer.html). For more information on Longformer's self-attention, please take a look at the corresponding [docs](https://huggingface.co/transformers/model_doc/longformer.html#longformer-self-attention). For summarization, we follow recommendations of the [paper](https://arxiv.org/abs/2004.05150) and use global attention only for the very first token. Finally, we make sure that no loss is computed on padded tokens by setting their index to `-100`.

In [53]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["article"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["abstract"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

For the sake of this notebook, we will reduce the training and validation data 
to a dummy dataset of sizes 250 and 25 respectively. For a full training run, those lines should be commented out.

Great, having defined the mapping function, let's preprocess the training data

In [54]:
# train_dataset = train_dataset.map(
#     process_data_to_model_inputs,
#     batched=True,
#     batch_size=batch_size,
#     remove_columns=["article", "abstract", "__index_level_0__"],
# )

and validation data

In [55]:
# val_dataset = val_dataset.map(
#     process_data_to_model_inputs,
#     batched=True,
#     batch_size=batch_size,
#     remove_columns=["article", "abstract", "__index_level_0__"],
# )

**train_dataset**  
attention mask  
global_attention_mask  
input_ids  
labels

Finally, the datasets should be converted into the PyTorch format as follows.

In [56]:
train_dataset

Dataset({
    features: ['__index_level_0__', 'abstract', 'article'],
    num_rows: 2445
})

In [57]:
# train_dataset.set_format(
#     type="torch",
#     columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
# )
# val_dataset.set_format(
#     type="torch",
#     columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
# )

Alright, we're almost ready to start training. Let's load the model via the `AutoModelForSeq2SeqLM` class.

In [58]:
from transformers import AutoModelForSeq2SeqLM

We've decided to stick to the smaller model `"allenai/led-base-16384"` for the sake of this notebook. In addition, we directly enable gradient checkpointing and disable the caching mechanism to save memory.

In [59]:
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)

During training, we want to evaluate the model on Rouge, the most common metric used in summarization, to make sure the model is indeed improving during training. For this, we set fitting generation parameters. We'll use beam search with a small beam of just 2 to save memory. Also, we force the model to generate at least 100 tokens, but no more than 512. In addition, some other generation parameters are set that have been found helpful for generation. For more information on those parameters, please take a look at the [docs](https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.generation_utils.GenerationMixin.generate).

In [60]:
# set generate hyperparameters
led.config.num_beams = 2
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

Next, we also have to define the function the will compute the `"rouge"` score during evalution.

Let's load the `"rouge"` metric from 🤗datasets and define the `compute_metrics(...)` function.

In [61]:
rouge = load_metric("rouge")

The compute metrics function expects the generation output, called `pred.predictions` as well as the gold label, called `pred.label_ids`.

Those tokens are decoded and consequently, the rouge score can be computed.

In [62]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

Now, we're ready to start training. Let's import the `Seq2SeqTrainer` and `Seq2SeqTrainingArguments`.

In [63]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In contrast to the usual `Trainer`, the `Seq2SeqTrainer` makes it possible to use the `generate()` function during evaluation. This should be enabled with `predict_with_generate=True`. Because our GPU RAM is limited, we make use of gradient accumulation by setting `gradient_accumulation_steps=4` to have an effective `batch_size` of 2 * 4 = 8.

Other training arguments can be read upon in the [docs](https://huggingface.co/transformers/main_classes/trainer.html?highlight=trainingarguments#transformers.TrainingArguments).

In [64]:
# # enable fp16 apex training
# training_args = Seq2SeqTrainingArguments(
#     predict_with_generate=True,
#     evaluation_strategy="steps",
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     fp16=True,
#     output_dir="./",
#     logging_steps=5,
#     eval_steps=10,
#     save_steps=10,
#     save_total_limit=2,
#     gradient_accumulation_steps=4, ## 줄일 수 있는 parameter인듯. default=4
#     num_train_epochs=1,
# )

The training arguments, along with the model, tokenizer, datasets and the `compute_metrics` function can then be passed to the `Seq2SeqTrainer`

In [65]:
# trainer = Seq2SeqTrainer(
#     model=led,
#     tokenizer=tokenizer,
#     args=training_args,
#     compute_metrics=compute_metrics,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
# )

and we can start training. This will take about ~35min.

In [66]:
# ! pip install apex

In [67]:
# trainer.train()

This completes the fine-tuning tutorial for LED. This training script with some small changes was used to train [this](https://huggingface.co/patrickvonplaten/led-large-16384-pubmed) checkpoint, called `" patrickvonplaten/led-large-16384-pubmed"` on a single GPU for ca. 3 days. Evaluating `" patrickvonplaten/led-large-16384-pubmed"` on Pubmed's test data gives a Rouge-2 score of **19.33** which is around 1 Rouge-2 point below SOTA performance on Pubmed.

In the Appendix below, the condensed training and evaluation scripts that were used locally to finetune `" patrickvonplaten/led-large-16384-pubmed"` are attached.

## Training

In [68]:
# ! pip3 install amp-atomistics

In [69]:
# # 시간 오래 걸림. 30분 정도 apex나 amp 사용 안하면 설치할 필요 없음

# !git clone https://github.com/NVIDIA/apex.git
# %cd apex
# !pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .

In [70]:
# import apex
# from apex import amp

In [71]:
# !nvidia-smi

In [72]:
import torch
torch.cuda

<module 'torch.cuda' from '/home/aiffelsummabot/anaconda3/envs/summabot/lib/python3.7/site-packages/torch/cuda/__init__.py'>

In [73]:
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

True


In [74]:
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

In [75]:
print('학습을 진행하는 기기:',device)

학습을 진행하는 기기: cuda:0


In [76]:
import torch
import math

print(torch.__version__) # torch version 출력

dtype = torch.float
# device = torch.device("cpu")
device = torch.device("cuda")

1.10.0+cu102


In [77]:
import os
import torch

os.environ["TOKENIZERS_PARALLELISM"] = "False"

In [78]:
# #!/usr/bin/env python3
# from datasets import load_dataset, load_metric
# from transformers import (
#     Seq2SeqTrainer,
#     Seq2SeqTrainingArguments,
#     AutoTokenizer,
#     AutoModelForSeq2SeqLM,
# )

# # load rouge
# rouge = load_metric("rouge")

# # load pubmed
# # pubmed_train = load_dataset("scientific_papers", "pubmed", ignore_verifications=True, split="train")
# # pubmed_val = load_dataset("scientific_papers", "pubmed", ignore_verifications=True, split="validation[:10%]")
# pubmed_train = train_dataset
# pubmed_val = val_dataset

# # comment out following lines for a test run
# # pubmed_train = pubmed_train.select(range(32))
# # pubmed_val = pubmed_val.select(range(32))

# # load tokenizer
# tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")


# # max encoder length is 8192 for PubMed
# encoder_max_length = 4096
# decoder_max_length = 512
# batch_size = 1


# def process_data_to_model_inputs(batch):
#     # tokenize the inputs and labels
#     inputs = tokenizer(
#         batch["article"],
#         padding="max_length",
#         truncation=True,
#         max_length=encoder_max_length,
#     )
#     outputs = tokenizer(
#         batch["abstract"],
#         padding="max_length",
#         truncation=True,
#         max_length=decoder_max_length,
#     )

#     batch["input_ids"] = inputs.input_ids
#     batch["attention_mask"] = inputs.attention_mask

#     # create 0 global_attention_mask lists
#     batch["global_attention_mask"] = len(batch["input_ids"]) * [
#         [0 for _ in range(len(batch["input_ids"][0]))]
#     ]

#     # since above lists are references, the following line changes the 0 index for all samples
#     batch["global_attention_mask"][0][0] = 1
#     batch["labels"] = outputs.input_ids

#     # We have to make sure that the PAD token is ignored
#     batch["labels"] = [
#         [-100 if token == tokenizer.pad_token_id else token for token in labels]
#         for labels in batch["labels"]
#     ]

#     return batch


# # # map train data
# # pubmed_train = pubmed_train.map(
# #     process_data_to_model_inputs,
# #     batched=True,
# #     batch_size=batch_size,
# #     remove_columns=["article", "abstract", "__index_level_0__"],
# # )

# # # map val data
# # pubmed_val = pubmed_val.map(
# #     process_data_to_model_inputs,
# #     batched=True,
# #     batch_size=batch_size,
# #     remove_columns=["article", "abstract", "__index_level_0__"],
# # )

# # # set Python list to PyTorch tensor
# # pubmed_train.set_format(
# #     type="torch",
# #     columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
# # )

# # # set Python list to PyTorch tensor
# # pubmed_val.set_format(
# #     type="torch",
# #     columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
# # )

# # enable fp16 apex training  ## ## name 'amp' is not defined 문제로 주석처리
# training_args = Seq2SeqTrainingArguments(
#     predict_with_generate=True,
#     evaluation_strategy="steps",
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     fp16=False, ## default = True
#     fp16_backend="auto", ## default = apex
#     output_dir="./LED_test",
#     logging_steps=250,
#     eval_steps=5000,
#     save_steps=500,
#     warmup_steps=1500,
#     save_total_limit=2,
#     gradient_accumulation_steps=4,
# )


# # compute Rouge score during validation
# def compute_metrics(pred):
#     labels_ids = pred.label_ids
#     pred_ids = pred.predictions

#     pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     labels_ids[labels_ids == -100] = tokenizer.pad_token_id
#     label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

#     rouge_output = rouge.compute(
#         predictions=pred_str, references=label_str, rouge_types=["rouge2"]
#     )["rouge2"].mid

#     return {
#         "rouge2_precision": round(rouge_output.precision, 4),
#         "rouge2_recall": round(rouge_output.recall, 4),
#         "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
#     }


# # load model + enable gradient checkpointing & disable cache for checkpointing
# led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)

# # set generate hyperparameters
# led.config.num_beams = 4
# led.config.max_length = 512
# led.config.min_length = 100
# led.config.length_penalty = 2.0
# led.config.early_stopping = True
# led.config.no_repeat_ngram_size = 3


# # instantiate trainer
# trainer = Seq2SeqTrainer(
#     model=led,
#     tokenizer=tokenizer,
#     args=training_args, ## optional
#     compute_metrics=compute_metrics,
#     train_dataset=pubmed_train,
#     eval_dataset=pubmed_val,
# )

# # start training
# with torch.autograd.set_detect_anomaly(True): ## RuntimeError 때문에 추가
#     trainer.train()

## Evaluation

In [79]:
train_dataset

Dataset({
    features: ['__index_level_0__', 'abstract', 'article'],
    num_rows: 2445
})

In [80]:
# test_dataset = test_dataset.map(
#     process_data_to_model_inputs,
#     batched=True,
#     batch_size=batch_size,
#     remove_columns=["article", "abstract" ]
# )

In [81]:
test_dataset

Dataset({
    features: ['abstract', 'article'],
    num_rows: 328
})

In [82]:
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel

main이 실행되는데 main은 다시 main_worker 들을 multi-processing으로 실행합니다. GPU 4개를 하나의 노드로 보고 world_size를 설정합니다. 그러면 mp.spawn 함수가 2개의 GPU에서 따로 따로 main_worker를 실행

In [83]:
import torch

from datasets import load_dataset, load_metric
from transformers import LEDTokenizer, LEDForConditionalGeneration

# load pubmed
pubmed_test = test_dataset

# load tokenizer
model_path = "/home/aiffelsummabot/LED/checkpoint-500/"
tokenizer = LEDTokenizer.from_pretrained(model_path)
model = LEDForConditionalGeneration.from_pretrained(model_path).to("cuda").half()


def generate_answer(batch):
    inputs_dict = tokenizer(batch["article"], padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
    input_ids = inputs_dict.input_ids.to("cuda")
    attention_mask = inputs_dict.attention_mask.to("cuda")
    global_attention_mask = torch.zeros_like(attention_mask)
    # put global attention on <s> token
    global_attention_mask[:, 0] = 1

    predicted_abstract_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
    batch["predicted_abstract"] = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)
    return batch


result = pubmed_test.map(generate_answer, batched=True, batch_size=4)

# load rouge
rouge = load_metric("rouge")

print("Result:", rouge.compute(predictions=result["predicted_abstract"], references=result["abstract"], rouge_types=["rouge2"])["rouge2"].mid)


/home/aiffelsummabot/anaconda3/envs/summabot/lib/python3.7/site-packages/transformers/generation_utils.py:1632: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size



Result: Score(precision=0.48448750906971416, recall=0.3208196580430346, fmeasure=0.3733898039246645)


In [89]:
result

Dataset({
    features: ['abstract', 'article', 'predicted_abstract'],
    num_rows: 328
})

In [94]:
import pandas as pd

predicted_result = pd.DataFrame({'abstract': result['abstract'],
                                                    'article': result['article'],
                                                    'predicted_abstract': result['predicted_abstract']})

predicted_result.to_csv('./predicted_document.csv')